In [1]:
#!pip install requests

In [2]:
#!pip install bs4

In [3]:
import time, datetime
import random
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3 as lite

number_of_points=10
url = 'https://www.t-online.de/finanzen/boerse/indizes/id_51947152/isin_DE0008469008/indizes.html'
db_name = 'Aktienkurse_DB.db'

fillDB=True

i=0
while i < number_of_points:
    t0=time.time()
    n = datetime.datetime.now()
    now = n.strftime('%d.%m.%Y %H:%M:%S')
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    type(html_soup)
    dax30_containers = html_soup.find_all('div', class_ = 'Tllc Tbd')
    target = dax30_containers[1]
    firma = []
    isin = []
    kurs = []
    for item in target.tbody.find_all('td', class_ = 'Tleft'):
        if item.find('b') is not None:
            firma.append(item.b.a.text)

    for item in target.tbody.find_all('td'):
        if item.find('a') is not None:
            isin.append(item.a.text)
        if item.find('b') is not None:
            kurs.append(item.b.text)
    
    kurs_n = [x[:-2].replace(',','.') for x in kurs if x not in firma]
    isin_n = [x for x in isin if x not in firma] 

    if fillDB:
        sheet='Daten'
        print ("Filling DB... ")
        conn = lite.connect(db_name)
        cur = conn.cursor()

        if i == 0:
            test_df = pd.DataFrame({'Firma': firma,
            'ISIN': isin_n,                       
            now: kurs_n,
            })
            #print(test_df)
            for item in ['Firma','ISIN',str(now)]:
                cur.execute('DROP TABLE IF EXISTS '+sheet)
                test_df.to_sql(sheet, conn, if_exists='replace', index=False) # - writes the pd.df to SQLIte DB
        else:
            test_df[now]=kurs_n
            cur.execute('DROP TABLE IF EXISTS '+sheet)
            test_df.to_sql(sheet, conn, if_exists='replace', index=False) # - writes the pd.df to SQLIte DB
        
    conn.commit()
    conn.close()
        
    time.sleep(random.randint(61,100))
    i+=1

Filling DB... 


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2532: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


Filling DB... 
Filling DB... 
Filling DB... 
Filling DB... 
Filling DB... 
Filling DB... 
Filling DB... 
Filling DB... 
Filling DB... 
